# Importing necessary libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import RMSprop

Using TensorFlow backend.


# Analyzing the given data

In [2]:
df = pd.read_csv('../input/train.csv')
df1 = pd.read_csv('../input/test.csv')
df2 = pd.read_csv('../input/sample_submission.csv')

In [3]:
df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# dividing the training data into feature matrix and label vector
X = df.drop(columns = ['label'])
y = df['label']

# Pre-processing

In [5]:
def preprocessing(X, y): # function to pre-process train and test data for fitting and prediction
    X = np.array(X)
    y = np.array(y)
    X = X.reshape(X.shape[0], 28, 28, 1)
    X = X/255
    y = keras.utils.to_categorical(y, num_classes = 10) # Converts each number in the label columns to vectors of 0s and 1, where the position of 1 signifies the value of that number.
    return X, y

In [6]:
X, y = preprocessing(X, y)

In [7]:
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0) # defining the optimizer for the loss function for the cnn defined below

# Defining the CNN model

In [8]:
# Defining the cnn model which will be used to fit the training data
model = Sequential()
model.add(Convolution2D(32, 3, data_format = 'channels_last', activation = 'relu', input_shape = (28, 28, 1)))
model.add(Convolution2D(32, 3, data_format = 'channels_last', activation = 'relu'))
model.add(Convolution2D(32, 3, data_format = 'channels_last', activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2, 2)))



model.add(Convolution2D(64, 3, data_format = 'channels_last', activation = 'relu'))
model.add(Convolution2D(64, 3, data_format = 'channels_last', activation = 'relu'))
model.add(Convolution2D(64, 3, data_format = 'channels_last', activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2, 2)))



model.add(Flatten())
model.add(Dense(256, activation = 'relu'))
model.add(Dense(256, activation = 'relu'))
model.add(Dense(256, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Activation('softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = optimizer, metrics = ['accuracy'])


In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 22, 22, 32)        9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 22, 22, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 11, 11, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 9, 9, 64)          18496     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 7, 7, 64)          36928     
__________

In [10]:
model.fit(X, y, epochs = 60) # fitting the training the data on the above defined model

Epoch 1/60
42000/42000 [==============================] - 15s 358us/step - loss: 0.2024 - acc: 0.9426
Epoch 2/60
42000/42000 [==============================] - 11s 268us/step - loss: 0.1077 - acc: 0.9755
Epoch 3/60
42000/42000 [==============================] - 11s 265us/step - loss: 0.0991 - acc: 0.9803
Epoch 4/60
42000/42000 [==============================] - 11s 270us/step - loss: 0.1063 - acc: 0.9814
Epoch 5/60
42000/42000 [==============================] - 11s 264us/step - loss: 0.1118 - acc: 0.9809
Epoch 6/60
42000/42000 [==============================] - 11s 266us/step - loss: 0.1174 - acc: 0.9826
Epoch 7/60
42000/42000 [==============================] - 11s 266us/step - loss: 0.1208 - acc: 0.9835
Epoch 8/60
42000/42000 [==============================] - 11s 266us/step - loss: 0.1206 - acc: 0.9832
Epoch 9/60
42000/42000 [==============================] - 11s 270us/step - loss: 0.1325 - acc: 0.9830
Epoch 10/60
42000/42000 [==============================] - 11s 268us/step - loss: 

# Predicting on test data over the fitted CNN model

In [11]:
X_test, y_dummy = preprocessing(df1, pd.Series(0))

In [12]:
y_pred = model.predict(X_test) # Predicting on the test data

In [13]:
# Since the predicted numbers are in the form of vectors constituting numbets between 0 and 1, the position of '1' is extracted from each vector as it represents the number itself
num = []
for i in range(len(y_pred)):
    for j in range(10):
        if y_pred[i][j] == y_pred[i].max():
            num.append(j)

In [14]:
y_pred = num

In [15]:
len(y_pred)

28000

# Preparing for submission

In [16]:
submission = pd.DataFrame({'ImageId':df2['ImageId'],'Label':y_pred}) # Preparing the submission data.
submission.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3


In [17]:
# Converting the submission data into a .csv file for scoring
filename = 'submission.csv'
submission.to_csv(filename, index = False)